import libraries(I provide all libs that I need when make this tasks, if you need some external import them here)

In [40]:
import findspark
findspark.init()
findspark.find()

'C:\\spark'

In [71]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
from pyspark.sql.functions import col
from pyspark.sql.functions import max, avg, min
from pyspark.sql.functions import lit
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number
from pyspark.sql.functions import when
import json

In [42]:
from pyspark.sql.functions import monotonically_increasing_id

create local SparkSession

In [43]:
spark = SparkSession.builder.master("local[1]") \
                    .appName('FirstSparkApp.com') \
                    .getOrCreate()

read csv with inferschema

In [44]:
df = spark.read.options(inferSchema=True, header=True).csv("ds_salaries.csv")

read csv one more time with the same code and you will see that it almostly don't take time, because info already in SparkSession and it will not read nothing
from this file

In [45]:
df = spark.read.options(inferSchema=True, header=True).csv("ds_salaries.csv")

write schema of scv on screen

In [46]:
df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- work_year: integer (nullable = true)
 |-- experience_level: string (nullable = true)
 |-- employment_type: string (nullable = true)
 |-- job_title: string (nullable = true)
 |-- salary: integer (nullable = true)
 |-- salary_currency: string (nullable = true)
 |-- salary_in_usd: integer (nullable = true)
 |-- employee_residence: string (nullable = true)
 |-- remote_ratio: integer (nullable = true)
 |-- company_location: string (nullable = true)
 |-- company_size: string (nullable = true)



In [47]:
schema = df.schema.json()
with open('schema.txt', 'w') as F:  
    json.dump(schema, F)

create schema of this scv

In [48]:
schema

'{"fields":[{"metadata":{},"name":"_c0","nullable":true,"type":"integer"},{"metadata":{},"name":"work_year","nullable":true,"type":"integer"},{"metadata":{},"name":"experience_level","nullable":true,"type":"string"},{"metadata":{},"name":"employment_type","nullable":true,"type":"string"},{"metadata":{},"name":"job_title","nullable":true,"type":"string"},{"metadata":{},"name":"salary","nullable":true,"type":"integer"},{"metadata":{},"name":"salary_currency","nullable":true,"type":"string"},{"metadata":{},"name":"salary_in_usd","nullable":true,"type":"integer"},{"metadata":{},"name":"employee_residence","nullable":true,"type":"string"},{"metadata":{},"name":"remote_ratio","nullable":true,"type":"integer"},{"metadata":{},"name":"company_location","nullable":true,"type":"string"},{"metadata":{},"name":"company_size","nullable":true,"type":"string"}],"type":"struct"}'

restart kernel without cleaning output and after restarting you need to initialize SparkSession, after initialize start execute only cells from cell with schema=
=StructType.... 
To restart kernel click Kernel, Restart.

read ds_salaries with predefined schema and compare results from this cell and cell with inferSchema

In [49]:
with open('schema.txt', 'r') as F:  
    saved_schema = json.load(F)
    
new_schema = StructType.fromJson(json.loads(saved_schema))
df = spark.read.options(inferSchema=True, header=True, schema=new_schema).csv("ds_salaries.csv")

this happens because read operation is lazy(transformation), but if you use inferschema it start to be action that will create Spark Job, because Spark need to loop throw all file to check datatypes for all columns and this can harm to your code(if we compare to parquet, it will also go to check data types, but parquet provide meta information, so Spark will not go throw all file, he will just read meta information, but csv don't provide such meta information). Also header make Spark to create one more Spark Job to check first line
to define name of columns and remember to skeep it when reading. Actual reading start when you will use first action. More about Spark Jobs you will see in next topic

write schema of scv on screen one more time and compare with previous

In [50]:
df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- work_year: integer (nullable = true)
 |-- experience_level: string (nullable = true)
 |-- employment_type: string (nullable = true)
 |-- job_title: string (nullable = true)
 |-- salary: integer (nullable = true)
 |-- salary_currency: string (nullable = true)
 |-- salary_in_usd: integer (nullable = true)
 |-- employee_residence: string (nullable = true)
 |-- remote_ratio: integer (nullable = true)
 |-- company_location: string (nullable = true)
 |-- company_size: string (nullable = true)



now continue to work with one of the dataframes that you create

print data in dataframe using df.show

In [51]:
df.show()

+---+---------+----------------+---------------+--------------------+--------+---------------+-------------+------------------+------------+----------------+------------+
|_c0|work_year|experience_level|employment_type|           job_title|  salary|salary_currency|salary_in_usd|employee_residence|remote_ratio|company_location|company_size|
+---+---------+----------------+---------------+--------------------+--------+---------------+-------------+------------------+------------+----------------+------------+
|  0|     2020|              MI|             FT|      Data Scientist|   70000|            EUR|        79833|                DE|           0|              DE|           L|
|  1|     2020|              SE|             FT|Machine Learning ...|  260000|            USD|       260000|                JP|           0|              JP|           S|
|  2|     2020|              SE|             FT|   Big Data Engineer|   85000|            GBP|       109024|                GB|          50|     

print data in dataframe using display(df.toPandas())

In [52]:
display(df.toPandas())

,_c0,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,0,2020,MI,FT,Data Scientist,70000,EUR,79833,DE,0,DE,L
1,1,2020,SE,FT,Machine Learning Scientist,260000,USD,260000,JP,0,JP,S
2,2,2020,SE,FT,Big Data Engineer,85000,GBP,109024,GB,50,GB,M
3,3,2020,MI,FT,Product Data Analyst,20000,USD,20000,HN,0,HN,S
4,4,2020,SE,FT,Machine Learning Engineer,150000,USD,150000,US,50,US,L
...,...,...,...,...,...,...,...,...,...,...,...,...
602,602,2022,SE,FT,Data Engineer,154000,USD,154000,US,100,US,M
603,603,2022,SE,FT,Data Engineer,126000,USD,126000,US,100,US,M
604,604,2022,SE,FT,Data Analyst,129000,USD,129000,US,0,US,M
605,605,2022,SE,FT,Data Analyst,150000,USD,150000,US,100,US,M


create df_job_title that consists from all job_titles without duplicates

In [53]:
df_job_title = df.select('job_title').distinct()

print all rows from df_job_titles without truncating jobs

In [54]:
df_job_title.show(10, False)

+-----------------------------+
|job_title                    |
+-----------------------------+
|3D Computer Vision Researcher|
|Lead Data Engineer           |
|Head of Machine Learning     |
|Data Specialist              |
|Data Analytics Lead          |
|Machine Learning Scientist   |
|Lead Data Analyst            |
|Data Engineering Manager     |
|Staff Data Scientist         |
|ETL Developer                |
+-----------------------------+
only showing top 10 rows



create  df_analytic that will consists from max, avg, min USD salaries for all job_titles using groupBy. name of fields is avg_salary, min_salary, max_salary

In [55]:
df_analytic = df.groupBy("job_title").agg(max("salary").alias("max_salary"), \
                                    min("salary").alias("min_salary"), \
                                    avg("salary").alias("avg_salary"))

print all rows from df_analytic without trancating jobs

In [56]:
df_analytic.show(truncate=False)

+----------------------------------------+----------+----------+------------------+
|job_title                               |max_salary|min_salary|avg_salary        |
+----------------------------------------+----------+----------+------------------+
|3D Computer Vision Researcher           |400000    |400000    |400000.0          |
|Lead Data Engineer                      |276000    |56000     |140333.33333333334|
|Head of Machine Learning                |6000000   |6000000   |6000000.0         |
|Data Specialist                         |165000    |165000    |165000.0          |
|Data Analytics Lead                     |405000    |405000    |405000.0          |
|Machine Learning Scientist              |260000    |12000     |158412.5          |
|Lead Data Analyst                       |1450000   |87000     |569000.0          |
|Data Engineering Manager                |174000    |51999     |119799.8          |
|Staff Data Scientist                    |105000    |105000    |105000.0    

now you need to add in df_analytic column row_id, that will show order of all job_titles depending on avg salary. they should be descending

In [57]:
df_new_analytic = df_analytic.sort(df_analytic.avg_salary.desc()).withColumn("index", monotonically_increasing_id())

print all data from df_analytic

In [58]:
df_new_analytic.show(20,False)

+-----------------------------+----------+----------+------------------+-----+
|job_title                    |max_salary|min_salary|avg_salary        |index|
+-----------------------------+----------+----------+------------------+-----+
|Head of Machine Learning     |6000000   |6000000   |6000000.0         |0    |
|ML Engineer                  |8500000   |14000     |2676666.6666666665|1    |
|BI Data Analyst              |11000000  |9272      |1902045.3333333333|2    |
|Lead Data Scientist          |3000000   |115000    |1101666.6666666667|3    |
|Data Science Manager         |7000000   |137141    |1062598.5833333333|4    |
|Lead Data Analyst            |1450000   |87000     |569000.0          |5    |
|Data Scientist               |30400000  |4000      |508347.2027972028 |6    |
|Big Data Engineer            |1672000   |18000     |455000.0          |7    |
|Data Analytics Lead          |405000    |405000    |405000.0          |8    |
|3D Computer Vision Researcher|400000    |400000    

it isn't beautifull, so we need to put now row_id on first place in df_analytic

In [59]:
df_new_analytic = df_new_analytic.select("index", "job_title", "max_salary", "min_salary", "avg_salary")

print df_analytic now

In [60]:
df_new_analytic.show(20,False)

+-----+-----------------------------+----------+----------+------------------+
|index|job_title                    |max_salary|min_salary|avg_salary        |
+-----+-----------------------------+----------+----------+------------------+
|0    |Head of Machine Learning     |6000000   |6000000   |6000000.0         |
|1    |ML Engineer                  |8500000   |14000     |2676666.6666666665|
|2    |BI Data Analyst              |11000000  |9272      |1902045.3333333333|
|3    |Lead Data Scientist          |3000000   |115000    |1101666.6666666667|
|4    |Data Science Manager         |7000000   |137141    |1062598.5833333333|
|5    |Lead Data Analyst            |1450000   |87000     |569000.0          |
|6    |Data Scientist               |30400000  |4000      |508347.2027972028 |
|7    |Big Data Engineer            |1672000   |18000     |455000.0          |
|8    |Data Analytics Lead          |405000    |405000    |405000.0          |
|9    |3D Computer Vision Researcher|400000    |4000

here you need to create df_exp_lvl with the biggest usd_salary(biggest_salary) for each experience_level(you need to save all fields like in entire dataframe)

In [61]:
df_exp_lvl = df.where("salary_currency == 'USD'").groupBy("experience_level").agg(max("salary").alias("max"))

In [62]:
display(df_exp_lvl.toPandas())

,experience_level,max
0,EX,600000
1,MI,450000
2,EN,250000
3,SE,412000


In [63]:
df_exp_lvl = df.join(df_exp_lvl, "experience_level")

In [64]:
df_exp_lvl.groupBy("max").count().orderBy(col("max").desc()).show(20,False)

+------+-----+
|max   |count|
+------+-----+
|600000|26   |
|450000|213  |
|412000|280  |
|250000|88   |
+------+-----+



print here df_exp_lvl

In [65]:
display(df_exp_lvl.toPandas())

,experience_level,_c0,work_year,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size,max
0,MI,0,2020,FT,Data Scientist,70000,EUR,79833,DE,0,DE,L,450000
1,SE,1,2020,FT,Machine Learning Scientist,260000,USD,260000,JP,0,JP,S,412000
2,SE,2,2020,FT,Big Data Engineer,85000,GBP,109024,GB,50,GB,M,412000
3,MI,3,2020,FT,Product Data Analyst,20000,USD,20000,HN,0,HN,S,450000
4,SE,4,2020,FT,Machine Learning Engineer,150000,USD,150000,US,50,US,L,412000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
602,SE,602,2022,FT,Data Engineer,154000,USD,154000,US,100,US,M,412000
603,SE,603,2022,FT,Data Engineer,126000,USD,126000,US,100,US,M,412000
604,SE,604,2022,FT,Data Analyst,129000,USD,129000,US,0,US,M,412000
605,SE,605,2022,FT,Data Analyst,150000,USD,150000,US,100,US,M,412000


create df_best that consists from rows where salary of guy same as biggest salary for other people in his exp_lvl and choose only columns: id, experience_level, biggest_salary, employee_residence

In [66]:
df_best = df_exp_lvl.filter("salary == max").select("_c0", "salary", "experience_level", "max", "employee_residence")

print df_best

In [67]:
display(df_best.toPandas())

,_c0,salary,experience_level,max,employee_residence
0,21,450000,MI,450000,IN
1,33,450000,MI,450000,US
2,37,250000,EN,250000,US
3,63,412000,SE,412000,US
4,97,450000,MI,450000,US
5,252,600000,EX,600000,US


drop duplicates if exist by experience_level

In [68]:
df_best = df_best.dropDuplicates(['experience_level'])

print df_best

In [69]:
display(df_best.toPandas())

,_c0,salary,experience_level,max,employee_residence
0,37,250000,EN,250000,US
1,252,600000,EX,600000,US
2,21,450000,MI,450000,IN
3,63,412000,SE,412000,US


create df_new_best from df_best without id, and make the next: when exp_level = MI we want middle, when SE we want senior, else Null

In [72]:
df_new_best = df_best.drop(col("_c0")).withColumn("position", when((df_best.experience_level == "MI"), lit("middle")) \
                              .when((df_best.experience_level == "SE"), lit("senior")) \
                              .otherwise(lit(None)) \
                  )

print df_new_best

In [73]:
df_new_best.show()

+------+----------------+------+------------------+--------+
|salary|experience_level|   max|employee_residence|position|
+------+----------------+------+------------------+--------+
|250000|              EN|250000|                US|    null|
|600000|              EX|600000|                US|    null|
|450000|              MI|450000|                IN|  middle|
|412000|              SE|412000|                US|  senior|
+------+----------------+------+------------------+--------+



write df_new_best like 1.csv and load then it to df_final

In [75]:
df_new_best.write.option("header", True).csv("1")

In [76]:
df_final = spark.read.csv("1\part-00000-a001cac7-331c-446c-bf2b-5bba3d4b4033-c000.csv", header=True)

print df_final

In [77]:
df_final.show()

+------+----------------+------+------------------+--------+
|salary|experience_level|   max|employee_residence|position|
+------+----------------+------+------------------+--------+
|250000|              EN|250000|                US|    null|
|600000|              EX|600000|                US|    null|
|450000|              MI|450000|                IN|  middle|
|412000|              SE|412000|                US|  senior|
+------+----------------+------+------------------+--------+



filter df_final to delete experience_level where it Null, then join this table by biggest_salary(salary_in_usd) and employee_residence with entire df

In [80]:
df_final = df_final.na.drop()

In [96]:
df = spark.read.options(inferSchema='True', header=True).csv("ds_salaries.csv")
final = df_final.join(df, (df_final.max == df.salary_in_usd) & (df_final.employee_residence == df.employee_residence), 'right')

print df_final

In [97]:
display(final.toPandas().iloc[:,:])

,salary,experience_level,max,employee_residence,position,_c0,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,None,None,None,None,None,0,2020,MI,FT,Data Scientist,70000,EUR,79833,DE,0,DE,L
1,None,None,None,None,None,1,2020,SE,FT,Machine Learning Scientist,260000,USD,260000,JP,0,JP,S
2,None,None,None,None,None,2,2020,SE,FT,Big Data Engineer,85000,GBP,109024,GB,50,GB,M
3,None,None,None,None,None,3,2020,MI,FT,Product Data Analyst,20000,USD,20000,HN,0,HN,S
4,None,None,None,None,None,4,2020,SE,FT,Machine Learning Engineer,150000,USD,150000,US,50,US,L
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
602,None,None,None,None,None,602,2022,SE,FT,Data Engineer,154000,USD,154000,US,100,US,M
603,None,None,None,None,None,603,2022,SE,FT,Data Engineer,126000,USD,126000,US,100,US,M
604,None,None,None,None,None,604,2022,SE,FT,Data Analyst,129000,USD,129000,US,0,US,M
605,None,None,None,None,None,605,2022,SE,FT,Data Analyst,150000,USD,150000,US,100,US,M


last task is to save in variable and then print this variable of the biggest salary_in_usd from df_final

In [93]:
final.agg(max("salary_in_usd")).show()

+------------------+
|max(salary_in_usd)|
+------------------+
|            600000|
+------------------+



In [103]:
final.explain('formatted')

== Physical Plan ==
AdaptiveSparkPlan (10)
+- == Final Plan ==
   * BroadcastHashJoin RightOuter BuildLeft (6)
   :- BroadcastQueryStage (4), Statistics(sizeInBytes=16.0 MiB, rowCount=2)
   :  +- BroadcastExchange (3)
   :     +- * Filter (2)
   :        +- Scan csv  (1)
   +- Scan csv  (5)
+- == Initial Plan ==
   BroadcastHashJoin RightOuter BuildLeft (9)
   :- BroadcastExchange (8)
   :  +- Filter (7)
   :     +- Scan csv  (1)
   +- Scan csv  (5)


(1) Scan csv 
Output [5]: [salary#1389, experience_level#1390, max#1391, employee_residence#1392, position#1393]
Batched: false
Location: InMemoryFileIndex [file:/C:/Андрей/spark_demo_course/1/part-00000-a001cac7-331c-446c-bf2b-5bba3d4b4033-c000.csv]
PushedFilters: [IsNotNull(max), IsNotNull(employee_residence)]
ReadSchema: struct<salary:string,experience_level:string,max:string,employee_residence:string,position:string>

(2) Filter [codegen id : 1]
Input [5]: [salary#1389, experience_level#1390, max#1391, employee_residence#1392, positio

It is the end of PySpark basics. In other lessons you will learn optimizations technics and how to make distributed system